* <span style="color: skyblue;">stuff 체인</span>: 이 체인은 모든 문서를 한 번에 LLM에 입력하여 답변을 생성하는 방식입니다. 짧은 텍스트 요약에 적합하며, 성능이 빠르지만 너무 긴 문서는 처리할 수 없는 한계가 있습니다.
* <span style="color: skyblue;">refine 체인</span>: 이 체인은 초기 응답을 생성한 후, 추가 문서를 보면서 점진적으로 개선하는 방식입니다. 긴 텍스트를 정리하거나 문서 간의 상호 참조가 필요한 경우에 유용합니다.

In [3]:
from langchain.chat_models import ChatOpenAI  # OpenAI의 챗 모델을 사용하기 위해 임포트
from langchain.document_loaders import UnstructuredFileLoader  # 비구조적 텍스트 파일을 로드하기 위한 모듈
from langchain.text_splitter import CharacterTextSplitter  # 텍스트를 특정 기준으로 나누기 위한 모듈
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings  # 임베딩과 캐시를 위한 모듈
from langchain.vectorstores import FAISS  # 벡터 저장소 FAISS를 사용하기 위한 모듈
from langchain.storage import LocalFileStore  # 로컬 파일 시스템에 데이터를 캐싱하기 위한 모듈
from langchain.chains import RetrievalQA  # 질의응답 체인(RetrievalQA)을 사용하기 위한 모듈
from langchain.document_loaders import TextLoader

# OpenAI 챗 모델 초기화
llm = ChatOpenAI()

# 캐시 데이터 저장을 위한 로컬 디렉터리 설정
cache_dir = LocalFileStore("./.cache/")

# 텍스트 분리 설정
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",  # 텍스트를 줄 단위로 분리
    chunk_size=600,  # 각 조각의 최대 크기
    chunk_overlap=100,  # 각 조각 간의 중복 영역 크기
)

# 텍스트 파일 로드 및 분리
loader = UnstructuredFileLoader("./document.txt")  # 문서 파일 로드
docs = loader.load_and_split(text_splitter=splitter)  # 텍스트를 splitter 기준으로 분리

# OpenAI 임베딩 생성
embeddings = OpenAIEmbeddings()

# 캐시 지원 임베딩 생성, 임베딩 결과를 캐싱하여 재사용
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 문서 데이터를 기반으로 FAISS 벡터 저장소 생성
vectorstore = FAISS.from_documents(docs, cached_embeddings)

# 질의응답 체인(RetrievalQA) 생성
chain = RetrievalQA.from_chain_type(
    llm=llm,  # OpenAI 언어 모델 사용
    chain_type="refine",  # 맵 기반 재정렬 체인 타입 지정
    retriever=vectorstore.as_retriever(),  # 벡터 저장소에서 검색 기능을 제공
)


# 체인 실행: 지정된 질문을 실행하고 결과 반환
chain.run("Describe Victory Mansions")  # "Victory Mansions를 묘사해줘"라는 질문 실행


'The description of Victory Mansions in George Orwell\'s novel "1984" as a run-down apartment complex takes on an even more bleak and oppressive tone when considered alongside the detailed portrayal of Winston\'s life within this setting. The dilapidated state of Victory Mansions serves as a physical manifestation of the oppressive regime ruling over Winston and the other residents. The constant surveillance, lack of basic necessities, and overall grim atmosphere of the apartment complex reflect the control and manipulation exerted by the ruling party.\n\nWinston\'s existence in Victory Mansions is marked by a sense of unease and fear, with every aspect of his life monitored and controlled. The details of his routine, such as the familiarity of the staff knowing his habits, the reserved corner table, and the undercharging for his drinks, further emphasize the suffocating atmosphere of the place. The fact that nobody wants to sit too close to Winston illustrates the isolation and aliena